In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Skewness Example") \
    .getOrCreate()

# Create a DataFrame with skewed data
data = [("A",), ("A",), ("A",), ("B",), ("B",), ("C",)]
columns = ["id"]
df = spark.createDataFrame(data, columns)
df.show()


24/03/29 10:55:32 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.25 instead (on interface wlp0s20f3)
24/03/29 10:55:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/29 10:55:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/29 10:55:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+---+
| id|
+---+
|  A|
|  A|
|  A|
|  B|
|  B|
|  C|
+---+



In [2]:
skewed_data = []
for i in range(1, 1000):
    skewed_data.extend(data)
skewed_df  = spark.createDataFrame(skewed_data, columns)
skewed_df.show()


+---+
| id|
+---+
|  A|
|  A|
|  A|
|  B|
|  B|
|  C|
|  A|
|  A|
|  A|
|  B|
|  B|
|  C|
|  A|
|  A|
|  A|
|  B|
|  B|
|  C|
|  A|
|  A|
+---+
only showing top 20 rows



In [3]:
# Union the original DataFrame with the skewed DataFrame
df = df.union(skewed_df)
df.show()
df1 = df

+---+
| id|
+---+
|  A|
|  A|
|  A|
|  B|
|  B|
|  C|
|  A|
|  A|
|  A|
|  B|
|  B|
|  C|
|  A|
|  A|
|  A|
|  B|
|  B|
|  C|
|  A|
|  A|
+---+
only showing top 20 rows



In [5]:
# Add a column to the DataFrame with the partition ID
df = df.withColumn("partitionId", spark_partition_id())
df.show()

+---+-----------+
| id|partitionId|
+---+-----------+
|  A|          1|
|  A|          2|
|  A|          3|
|  B|          5|
|  B|          6|
|  C|          7|
|  A|          8|
|  A|          8|
|  A|          8|
|  B|          8|
|  B|          8|
|  C|          8|
|  A|          8|
|  A|          8|
|  A|          8|
|  B|          8|
|  B|          8|
|  C|          8|
|  A|          8|
|  A|          8|
+---+-----------+
only showing top 20 rows



In [6]:
# Compute the number of records in each partition and sort the results
df = df.groupBy("partitionId").count().orderBy("partitionId", ascending=False)
df.show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|         15|  751|
|         14|  749|
|         13|  749|
|         12|  749|
|         11|  749|
|         10|  749|
|          9|  749|
|          8|  749|
|          7|    1|
|          6|    1|
|          5|    1|
|          3|    1|
|          2|    1|
|          1|    1|
+-----------+-----+



In [7]:
re_df = df1.repartition(10)
re_df = re_df.withColumn("part", spark_partition_id())
# re_df = re_df.groupBy("partitionId").count().orderBy("partitionId", ascending=False)


re_df = re_df.groupBy("part").count().orderBy("part",ascending=True)
re_df.show()
re_df.rdd.getNumPartitions()

+----+-----+
|part|count|
+----+-----+
|   0|  600|
|   1|  600|
|   2|  601|
|   3|  599|
|   4|  599|
|   5|  600|
|   6|  600|
|   7|  601|
|   8|  600|
|   9|  600|
+----+-----+



10